In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests
import gc
from tqdm import tqdm

pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


pd.options.display.max_columns = 100
pd.options.display.max_rows=1000


In [2]:
lct = "/Users/nam-yeong/Downloads/balso_tmp"
folders = [file for file in os.listdir(lct) if file.endswith('.csv')]
folders.sort()
for i in folders:
    print(os.path.join(lct,i))
    


/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-08_00_00_00+00_00-2024-04-09_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_33b25716-bac1-47e6-b040-be837fbb5ce6.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-09_00_00_00+00_00-2024-04-10_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_495ee3b4-e43f-422d-84d1-e0ec813345dd.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-10_00_00_00+00_00-2024-04-11_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_05214fdf-838f-4c99-a0ed-6b6d2b5fdb0d.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-11_00_00_00+00_00-2024-04-12_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_c662fe3f-b9e2-48dc-be8c-c4d35252edec.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-12_00_00_00+00_00-2024-04-13_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_79232362-6c0b-42d1-b3f8-6c5d8c927a38.csv
/Users/nam-yeong/Downloads/balso_tmp/balso_2024-04-13_00_00_00+00_00-2024-04-14_00_00_00+00_00_balsohecto@gmail.com_20240421-0617_2c7ec2ba-edd6-4

Clicl에 상품정보 담기지 않은 개발 이슈. Impression > CLick 퍼널 이어진 데이터를 선별해서 상품 정보 Stitching

In [18]:
tst = folders[-1]
# chunksize = 5000
# nrows = 1000
df = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{tst}")
df.query("`Event Category`.str.contains('AD')")
conv_users = list(set(df.query("`Event Category`.str.contains('Click')")['Airbridge Device ID'].tolist()))
df = df.query("`Airbridge Device ID` in @conv_users")
df[(df['Event Category'].str.contains('Impression') )& (df['Event Datetime']>='2024-04-08T23:59:50+09:00')]

df.drop_duplicates(subset=['Airbridge Device ID','Event Datetime','Event Category','Product List'],inplace=True)


In [21]:
len(df[df['Event Category'].str.contains('Click')])

57458

In [19]:
df.sort_values(by=['Airbridge Device ID', 'Event Datetime','Event Category'],ascending=[True,True,False], inplace=True)
df['prev_prop'] = df.groupby('Airbridge Device ID')['Product Name'].shift(1)
# df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]
df = df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]

df

,Event Category,Event Action,Event Label,Event Value,Event Date,Event Datetime,Semantic Event Properties,Custom Event Properties,Total Revenue,Total Revenue (Original),Currency,Currency (Original),Product List,Product ID,Product Name,Product Price,Product Quantity,Product Currency,Product Position,Product Brand ID,Product Brand Name,Product Category ID,Product Category Name,Airbridge Device ID,Platform,User ID,prev_prop
403757,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T19:29:18+09:00,"{""products"":[{""name"":""프리미엄 인도산 강황정 강황환 HACCP 인...",{},NaN,NaN,NaN,NaN,"[{""name"":""프리미엄 인도산 강황정 강황환 HACCP 인증 500mg 300정...",NaN,"프리미엄 인도산 강황정 강황환 HACCP 인증 500mg 300정, 150g, 2개",42300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0002CD6F-4106-464E-89BD-C726A1A2879D,iOS,caa65f1e-2436-4c04-a8da-78bd3968aab7,"프리미엄 인도산 강황정 강황환 HACCP 인증 500mg 300정, 150g, 2개"
65800,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T23:26:13+09:00,"{""products"":[{""name"":""테디베리 국내산 딸기청 1+1, 550g, ...",{},NaN,NaN,NaN,NaN,"[{""name"":""테디베리 국내산 딸기청 1+1, 550g, 2병"",""price"":...",NaN,"테디베리 국내산 딸기청 1+1, 550g, 2병",17820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00034858-1022-4103-AE9D-AEA98E20A495,iOS,38855c94-50e5-4142-aa98-8593171f59ea,"테디베리 국내산 딸기청 1+1, 550g, 2병"
688634,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T13:04:52+09:00,"{""products"":[{""name"":""본필 6인치 작업안전화 BSS805 발이편한...",{},NaN,NaN,NaN,NaN,"[{""name"":""본필 6인치 작업안전화 BSS805 발이편한 작업화, 1개"",""p...",NaN,"본필 6인치 작업안전화 BSS805 발이편한 작업화, 1개",39800.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0003a68a-7269-47c2-ba1a-3c47d6044103,Android,cc593bfe-2ff5-414c-95c6-7aca3b043240,"본필 6인치 작업안전화 BSS805 발이편한 작업화, 1개"
814660,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T04:51:23+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00069c4c-3eb0-4efc-a949-96ffda9cb8ef,Android,4bd4896f-f23e-44dd-bf54-ec084ef1f7be,두부김치만두 소포장 묶음 느끼하지 않고 담백한 강원도식 만두 먹어도 먹어도 질리지 ...
814654,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T04:51:27+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00069c4c-3eb0-4efc-a949-96ffda9cb8ef,Android,4bd4896f-f23e-44dd-bf54-ec084ef1f7be,곰곰 소중한 우리쌀밥
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736015,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T11:20:57+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff1c7de-1495-488d-89e2-ea3295d11ebd,Android,18d10bca-7631-4341-9156-9a14c2058aec,[HOTAX] 세련된 클래식 접이식 미니벨로 자전거 출퇴근 경량 미니 폴딩 바구니 ...
262376,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T21:24:51+09:00,"{""products"":[{""name"":""프리미엄 로즈힙 퓨레 엘라스틴 600mg 6...",{},NaN,NaN,NaN,NaN,"[{""name"":""프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘...",NaN,"프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘라스틴 60정 2병",31330.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff5e906-06ea-44ed-ae01-c4c3e6c4a821,Android,66482903-6256-4092-9d20-1e220804f289,"프리미엄 로즈힙 퓨레 엘라스틴 600mg 60정, 로즈힙 퓨레 엘라스틴 60정 2병"
497648,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T17:52:33+09:00,"{""products"":[{""name"":""여성 봄 정장 자켓"",""price"":6300...",{},NaN,NaN,NaN,NaN,"[{""name"":""여성 봄 정장 자켓"",""price"":63000}]",NaN,여성 봄 정장 자켓,63000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fff8f4f4-8256-4df3-a923-fef6b12a090d,Android,047d6ed4-a627-4d73-bc59-1467313bfde3,여성 봄 정장 자켓
243776,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-19,2024-04-19T21:37:36+09:00,"{""products"":[{""name"":""메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹...",{},NaN,NaN,NaN,NaN,"[{""name"":""메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정,...",NaN,"메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정, 다크브라운, 3개",29100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fffaeb83-177a-4747-bb1d-61b9e9377010,Android,ff8d11c2-4265-4f9c-9388-80790643f799,"메이시크 롱픽스 핑거팁 아이브로우 틴트 눈썹 타투펜+가이드 증정, 다크브라운, 3개"


In [22]:
len(df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]) 

57458

In [20]:
df.to_parquet(f'./{folders[i]}_conv.parquet')

자정에 데이터 끊기는 이슈. 타겟 date, d+1 데이터 로드해서 처리 후, d+1 데이터는 burn \
마지막 데이터의 경우, Click에 상품정보 포함되어 있음. 따라서 burn 이후, 정상 데이터 merge

In [17]:

for i in tqdm(range(len(folders))):
    if i < len(folders):
        
        tg = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{folders[i]}")
        tmp = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{folders[i+1]}")
        tmp = tmp[(tmp['Event Category'].str.contains('Clicks')) & (tmp['Event Datetime'] < '2024-04-09T00:30:00+09:00')]
        df = pd.concat([tg,tmp],axis=0)
        
        del tmp, tg
        gc.collect()
    else:
        df = pd.read_csv(f"/Users/nam-yeong/Downloads/balso_tmp/{folders[i]}")
        
    conv_users = list(set(df.query("`Event Category`.str.contains('Click')")['Airbridge Device ID'].tolist()))
    df = df.query("`Airbridge Device ID` in @conv_users")
    df.drop_duplicates(subset=['Airbridge Device ID','Event Datetime','Event Category','Product List'],inplace=True)
    
    print("전 클릭 전환 : ",len(df[df['Event Category'].str.contains('Click')]))
    
    df.sort_values(by=['Airbridge Device ID', 'Event Datetime','Event Category'],ascending=[True,True,False], inplace=True)
    df['prev_prop'] = df.groupby('Airbridge Device ID')['Product Name'].shift(1)
    df = df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]
    
    print("후 클릭 전환 : ",len(df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]))
    print("유실율 : ",len(df[(df['prev_prop'].notna()) & (df['Event Category'].str.contains('Click'))]) / len(df[df['Event Category'].str.contains('Click')]))
    
    df.to_parquet(f'./{folders[i]}_conv.parquet')
        

  0%|          | 0/12 [00:00<?, ?it/s]

전 클릭 전환 :  124142
후 클릭 전환 :  121547
유실율 :  1.0


  8%|▊         | 1/12 [02:38<29:01, 158.30s/it]

전 클릭 전환 :  106669
후 클릭 전환 :  105124
유실율 :  1.0


 17%|█▋        | 2/12 [06:13<31:56, 191.61s/it]

...

전 클릭 전환 :  121020
후 클릭 전환 :  118663
유실율 :  1.0


 83%|████████▎ | 10/12 [38:25<07:34, 227.41s/it]

전 클릭 전환 :  70993
후 클릭 전환 :  70218
유실율 :  1.0


 92%|█████████▏| 11/12 [45:52<04:10, 250.25s/it]


IndexError: list index out of range

In [16]:
pd.concat([df.head(10),df.tail(5)])


,Event Category,Event Action,Event Label,Event Value,Event Date,Event Datetime,Semantic Event Properties,Custom Event Properties,Total Revenue,Total Revenue (Original),Currency,Currency (Original),Product List,Product ID,Product Name,Product Price,Product Quantity,Product Currency,Product Position,Product Brand ID,Product Brand Name,Product Category ID,Product Category Name,Airbridge Device ID,Platform,User ID
4,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,855a9351-acea-4d20-836d-9b998a0b041e,Android,NaN
5,Ad Impression (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[{""name"":""재로우 자로-도필러스 EPS 유산균 100억...",{},NaN,NaN,NaN,NaN,"[{""name"":""재로우 자로-도필러스 EPS 유산균 100억 베지 캡, 60정, ...",NaN,"재로우 자로-도필러스 EPS 유산균 100억 베지 캡, 60정, 1개",21808.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,AA6A072E-248B-410A-8AF0-44161B2168CA,iOS,e968a486-0e67-4bb8-bd02-8af452e8755d
7,Ad Impression (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[{""name"":""생활 속 회계 / 오래"",""position""...",{},NaN,NaN,NaN,NaN,"[{""name"":""생활 속 회계 / 오래"",""position"":0,""price"":1...",NaN,생활 속 회계 / 오래,13500.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,855a9351-acea-4d20-836d-9b998a0b041e,Android,NaN
8,Ad Impression (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[{""name"":""라입 약산성 두피케어 대용량 리페어 퍼퓸 샴...",{},NaN,NaN,NaN,NaN,"[{""name"":""라입 약산성 두피케어 대용량 리페어 퍼퓸 샴푸 오션블루, 1개, ...",NaN,"라입 약산성 두피케어 대용량 리페어 퍼퓸 샴푸 오션블루, 1개, 1000ml",10320.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,ada52a4d-58a5-4045-ba2a-dbac4055b5b3,Android,NaN
9,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ada52a4d-58a5-4045-ba2a-dbac4055b5b3,Android,NaN
11,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8934c965-ff73-458b-b43d-68fe28706251,Android,NaN
13,Ad Impression (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[{""name"":""센트바스켓 플라워 비누꽃 장미 돈꽃다발 20...",{},NaN,NaN,NaN,NaN,"[{""name"":""센트바스켓 플라워 비누꽃 장미 돈꽃다발 20송이, 코랄, 1개"",...",NaN,"센트바스켓 플라워 비누꽃 장미 돈꽃다발 20송이, 코랄, 1개",23500.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,8934c965-ff73-458b-b43d-68fe28706251,Android,NaN
16,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0310ce98-0923-45ff-940d-87e3f9fa4af0,Android,NaN
18,Ad Impression (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[{""name"":""터치 고농축 액체세제 코델리아 3.05L, ...",{},NaN,NaN,NaN,NaN,"[{""name"":""터치 고농축 액체세제 코델리아 3.05L, 4개, 3.05L, 4...",NaN,"터치 고농축 액체세제 코델리아 3.05L, 4개, 3.05L, 4개",20500.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,4d347f24-b568-4a16-a941-f697886dec9f,Android,NaN
19,Ad Click (App),직광고_쿠팡,홈_상단_캐시버튼,NaN,2024-04-09,2024-04-09T23:59:59+09:00,"{""products"":[]}",{},NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4d347f24-b568-4a16-a941-f697886dec9f,Android,NaN
